## Prerequisites
- Nerlnet 1.6.0 is built (`./NerlnetBuild.sh --infra torch`).
- Synthetic CSV is synced to `/tmp/nerlnet/data/NerlnetData-master/nerlnet/synthetic_norm/synthetic_full.csv`.
- The example JSONs (`exp_torch_s1_w4.json`, `dc_torch_s1_w4.json`, `conn_torch_s1_w4.json`) remain inside `examples/torch/s1_w4_syntetic`.

## 1. Generate the TorchScript model
The following cell recreates the placeholder perceptron that matches the JSON metadata and writes it to `examples/torch/s1_w4_syntetic/models/synthetic_torch_example.pt`.

In [ ]:
from pathlib import Path
import subprocess
import sys

output_path = Path("examples/torch/s1_w4_syntetic/models/synthetic_torch_example.pt").resolve()
output_path.parent.mkdir(parents=True, exist_ok=True)
cmd = [
    sys.executable,
    "tests/scripts/generate_torch_test_model.py",
    "--output", str(output_path),
    "--samples", "2000",
    "--epochs", "400",
    "--lr", "0.005"
]
print("Creating TorchScript model at", output_path)
subprocess.run(cmd, check=True)

## 2. Inspect the distributed configuration
This helper cell visualizes the Torch worker definition and confirms the per-batch metadata that the worker expects during training.

In [ ]:
import json
from pprint import pprint

with open("examples/torch/s1_w4_syntetic/dc_torch_s1_w4.json", "r", encoding="utf-8") as handle:
    dc = json.load(handle)
workers = dc["workers"]
train_params = dc["model_sha"]["torch_s1w4_v160"]["train_params"]
print("Workers bound to the example model:")
pprint(workers)
print("
Torch training parameters:")
pprint(train_params)

## 3. Stage the JSONs and launch Nerlnet
1. Copy or symlink the JSON files into the paths referenced by `config/jsonsDir.nerlconfig` and `config/subnets.nerlconfig` (for example, point `dc.json` to `examples/torch/s1_w4_syntetic/dc_torch_s1_w4.json`).
2. Ensure `conn.json` resolves to `examples/torch/s1_w4_syntetic/conn_torch_s1_w4.json` and the experiment description is reachable at `examples/torch/s1_w4_syntetic/exp_torch_s1_w4.json`.
3. Build or rebuild the native bridges if anything changed (`./NerlnetBuild.sh --infra all`).
4. Launch the Erlang application via `./NerlnetRun.sh` in one terminal and keep it running.

## 4. Kick off the Torch full-flow
With Nerlnet running, trigger the example from a second terminal:
```bash
tests/NerlnetFullFlowTorchTest.sh --manual-start \
  --experiment examples/torch/s1_w4_syntetic/exp_torch_s1_w4.json \
  --dc examples/torch/s1_w4_syntetic/dc_torch_s1_w4.json \
  --conn examples/torch/s1_w4_syntetic/conn_torch_s1_w4.json
```
As batches stream through `s1`, each worker (`w1`-`w4`) stays active on the Torch bridge, preserving optimizer momentum and weights between messages.
Once both training and prediction phases finish, inspect `/tmp/nerlnet/results/` for anomaly summaries and `/tmp/nerlnet/torch/models/pt/` for updated checkpoints.

## 5. Analyze outcomes
Review `test_torch.log` (or the CSVs saved under `/tmp/nerlnet/results/.../model_perf.csv`) to compare Torch metrics against the OpenNN baseline. If the anomaly scores diverge, double-check that the generated TorchScript file matches the shapes advertised in `train_params` and that the JSON paths inside `config/jsonsDir.nerlconfig` are up to date.